# Getting started with Brightway

##### Get Ecoinvent account login details from a different file
A common approach is to store such confidential information in a file called `.env` in your project folder. Env stands for (project) environment, and the leading dot in the file name tells your operating system to treat this as a "hidden file". Your `.env` file will be a collection of key-value pairs, separated by an equation sign. For example, it might contain two lines such as:

```bash
ecoinvent_username=utrecht@university.nl
ecoinvent_password=super-duper-safe-password
project_name=Bike LCA project
```

We can then access this information with the `dotenv` and Python's built-in `os` libraries. The cell below contains a pretty standard code snippet that you could reuse in your projects.

Note: if you use Git (e.g GitHub) to manage your project, you will want to add `.env` in your `.gitignore` file to prevent publishing your credentials (e.g. on GitHub).

In [3]:
import os
from dotenv import load_dotenv  # To read the contents of the .env file

load_dotenv(override=True)
ECOINVENT_USERNAME = os.getenv("ecoinvent_username")
ECOINVENT_PASSWORD = os.getenv("ecoinvent_password")
PROJECT_NAME = os.getenv("project_name", "Bike LCA project")

In [4]:
PROJECT_NAME

'copernicus_brightway_tutorial'

## Initialization

In [5]:
# Brightway package for data management
import bw2data

### LCA project

> A Brightway project is an isolated set of data sufficient to do LCA calculations.
> 
> Source: https://docs.brightway.dev/en/latest/content/overview/glossary.html#term-Project

In [6]:
# List available project, in alphabetic order
sorted(bw2data.projects)

[Project: Bicycle example,
 Project: copernicus_brightway_tutorial,
 Project: default,
 Project: ecoinvent-3.12-cutoff,
 Project: mermaid_graph]

In [7]:
# Select (or create if not yet available) the project
bw2data.projects.set_current(name=PROJECT_NAME)

In [8]:
# Verify the current project name
bw2data.projects.current

'copernicus_brightway_tutorial'

### Databases

> A Brightway database is a **container for inventory** nodes. Databases are primarily organization tools, and their precise boundaries are determined by practitioners. Nodes must be associated with one and only one database, but edges can and often do cross from one database to another.
> 
> Brightway does not make a distinction between foreground and background databases.
> 
> Source: https://docs.brightway.dev/en/latest/content/overview/glossary.html#term-Database

In [9]:
# List databases available in the project
bw2data.databases

Databases dictionary with 3 object(s):
	bike
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

#### Import Ecoinvent database
Importing Ecoinvent (or other databases) into our Brightway project can be done in a couple of different ways. Here we show the most common way, which is to download it from Ecoinvent's server directly into our project. This requires that we provide our login credentials (username and password).

While it's possible to enter your credentials in-code, we strongly recommend against this. Imagine that you want to share your script with someone or submit it to an open-access journal: your license will be exposed! 

Once we obtained our Ecoinvent username and password, we're ready to import Ecoinvent into our Brightway project. We use Brightway's built-in `import_ecoinvent_release()` function.

In [10]:
# Brightway package for importing and exporting data
import bw2io

In [11]:
# Import ecoinvent (if not already)
if "ecoinvent-3.11-cutoff" not in bw2data.databases:
    bw2io.import_ecoinvent_release(
        version='3.11',
        system_model='cutoff',
        username=ECOINVENT_USERNAME,
        password=ECOINVENT_PASSWORD,
    )

In [12]:
# Make EcoInvent database usable via a Python object
ecoinvent = bw2data.Database('ecoinvent-3.11-cutoff')

## Modelling the LCA of a simple Ecoinvent activity

### Selecting an activity

Imagine that you want to calculate the GHG footprint of 1 kWh of low voltage electricity in the Netherlands.

We can look on Ecoinvent's ecoQuery if such activity exists and how it is called: https://ecoquery.ecoinvent.org/

We will use this one: https://ecoquery.ecoinvent.org/3.11/cutoff/dataset/7105/documentation

In [13]:
# Searching an activity in the database. Here it is Ecoinvent, but it could be any other database in the current project
activities = ecoinvent.search("market for electricity, low voltage")
# There are many activities with this name
activities

['electricity voltage transformation from medium to low voltage, renewable energy products' (kilowatt hour, CH, None),
 'market group for electricity, low voltage' (kilowatt hour, RAF, None),
 'market group for electricity, low voltage' (kilowatt hour, BR, None),
 'market group for electricity, low voltage' (kilowatt hour, CN, None),
 'market group for electricity, low voltage' (kilowatt hour, CA, None),
 'market group for electricity, low voltage' (kilowatt hour, IN, None),
 'market group for electricity, low voltage' (kilowatt hour, RER, None),
 'market group for electricity, low voltage' (kilowatt hour, RAS, None),
 'market group for electricity, low voltage' (kilowatt hour, CN-SGCC, None),
 'market group for electricity, low voltage' (kilowatt hour, RME, None),
 'market group for electricity, low voltage' (kilowatt hour, GLO, None),
 'market group for electricity, low voltage' (kilowatt hour, RNA, None),
 'market group for electricity, low voltage' (kilowatt hour, US, None),
 'mark

In [ ]:
# Let's refine it with the country code of the Netherlands 'NL'
activities = ecoinvent.search("market for electricity, low voltage, NL")
# There are still some activities with this name
activities

['electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, NL, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, NL, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, NL, None),
 'market for electricity, low voltage' (kilowatt hour, NL, None),
 'electricity, low voltage, residual mix' (kilowatt hour, NL, None),
 'market for electricity, medium voltage' (kilowatt hour, NL, None),
 'market for electricity, for reuse in municipal waste incineration only' (kilowatt hour, NL, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, NL, None),
 'market for electricity, high voltage' (kilowatt hour, NL, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, NL, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, NL, None),
 'electricity 

In [23]:
# Selecting the right one
activity_low_voltage_electricity_NL = ecoinvent.get(
    name="market for electricity, low voltage", 
    location="NL",
)

### Selecting the LCIA indicator

#### List available LCIA methods and indicators

In [25]:
# List all available methods in the current project
lcia_methods = bw2data.methods
list(lcia_methods)

[('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('ecoinvent-3.11',
  'CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP 

#### Selecting one LCIA indicator

In [26]:
# Select climate change indicators from EF 3.1 LCIA method that is provided by Ecoinvent
ef3_1_climate_change_lcia_indicator = [
    m 
    for m in lcia_methods 
    if m[0] == "ecoinvent-3.11" 
    and "climate change" == m[2] 
    and "EF v3.1" == m[1]
][0]
ef3_1_climate_change_lcia_indicator

('ecoinvent-3.11',
 'EF v3.1',
 'climate change',
 'global warming potential (GWP100)')

### Setting up the LCA

In [27]:
# Brightway package to compute LCAs or solve LCIs
import bw2calc

/Users/gergo/venvs/brightway-stable/.venv/lib/python3.13/site-packages/bw2calc/__init__.py:53: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


In [ ]:
# Build the LCA
lca = bw2calc.LCA(
    demand={activity_low_voltage_electricity_NL: 1}, 
    method=ef3_1_climate_change_lcia_indicator,
)

### Running the LCA

In [29]:
# Solve the LCI, necessary to compute the LCA.
# TODO: explain or validate: It calculates the sum of each of the elementary flows due to the functional unit, across the different databases (here only Ecoinvent) 
lca.lci()

In [30]:
# Calculate the LCIA results based on the LCI and the characterization factors from the LCIA method
lca.lcia()

#### Final result

In [31]:
# Output our final result
lca.score

0.3970655748216489

In [ ]:
# With the right unit
lca.score

0.3970655748216489

In [33]:
type(bw2data.Method(ef3_1_climate_change_lcia_indicator))
bw2data.Method(ef3_1_climate_change_lcia_indicator)

Brightway2 Method: ecoinvent-3.11: EF v3.1: climate change: global warming potential (GWP100)

## Import LCI via Excel

### Importing and matching

In [55]:
# Safety first. We recreate the LCI every time to avoid errors
if 'bike-db' in bw2data.databases:
    del bw2data.databases['bike-db']

In [107]:
# Import LCI from Excel file
excel_lci = bw2io.ExcelImporter("bike-database.xlsx")

Extracted 1 worksheets in 0.02 seconds


In [83]:
# This method is some default data processing that must be called right after importing from Excel
excel_lci.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.31 seconds


The `match_database` method does the heavy lifting of translating the Excel file (e.g. spreadsheet) to Brightway's data structure for databases (e.g. graph). "Matching" means establishing and connecting activities and exchanges in a way that Brightway understands them. We call this method as many times as many databases our LCI connects to, including itself (for internal exchanges).

In [89]:
# Matching the LCI to itself, i.e. activities and exchanges internal to the LCI
excel_lci.match_database(fields=["name", "reference product", "location"])


Applying strategy: link_iterable_by_fields


In [90]:
# Matching the LCI to ecoinvent (technosphere)
excel_lci.match_database(
    "ecoinvent-3.11-cutoff", fields=["name", "unit", "location", "reference product"]
)

Applying strategy: link_iterable_by_fields


In [91]:
# Matching the LCI to ecoinvent (biosphere)
excel_lci.match_database(
    "ecoinvent-3.11-biosphere", fields=["name", "categories", "location"]
)

Applying strategy: link_iterable_by_fields


In [104]:
# Get the overview of our LCI in Brightway. Check if there are unlinked exchanges
excel_lci.statistics()

Graph statistics for `bike-db` importer:
2 graph nodes:
	processwithreferenceproduct: 2
6 graph edges:
	technosphere: 4
	biosphere: 2
6 edges to the following databases:
	ecoinvent-3.11-cutoff: 3
	ecoinvent-3.11-biosphere: 2
	bike-db: 1
0 unique unlinked edges (0 total):




(2, 6, 0, 0)

In [105]:
excel_lci.write_database()

100%|██████████| 2/2 [00:00<00:00, 843.16it/s]

12:01:36+0100 [info     ] Vacuuming database            


Created database: bike-db


### Using the LCI

In [94]:
bw2io.databases

Databases dictionary with 3 object(s):
	bike-db
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

In [ ]:
bike_db = bw2data.Database("bike-db")

In [96]:
for act in bike_db:
    print(act)

'Carbon fibre production' (kilogram, NL, None)
'Bike production' (unit, NL, None)


In [97]:
bike = bike_db.get(name="Bike production")

In [ ]:
# Build the LCA as an object
lca = bw2calc.LCA(
    demand={bike: 1}, 
    method=ef3_1_climate_change_lcia_indicator,
)

### Running the LCA

In [ ]:
# Solve the LCI, necessary to compute the LCA.
lca.lci()

In [100]:
# Calculate the LCIA results based on the LCI and the characterization factors from the LCIA method
lca.lcia()

#### Final result

In [101]:
# Output our final result
lca.score

232.83130767826992